In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import joblib
import os

In [3]:
cd ..

/Users/yunxizzy/code/theresa-sporn/WorkforceSentimentMonitoring


In [4]:
ls

MANIFEST.in                            lexicon/
Makefile                               notebooks/
Procfile                               raw_data/
README.md                              requirements.txt
WorkforceSentimentMonitoring/          scripts/
WorkforceSentimentMonitoring.egg-info/ setup.py
app.py                                 setup.sh
build/                                 test_data/
dist/                                  tests/
joblib_files/


# Redo mnb feature extraction

In [88]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [6]:
# load train test data
from WorkforceSentimentMonitoring.data import get_prepaired_data

In [7]:
X_train, X_test, y_train, y_test = get_prepaired_data()

Reading data...
Merging data into a single DataFrame...
Dropping initial text columns...
Identifying entries in other languages...



Drop 437 entries? [y] / n
y
Dropping 437 entries...
Splitting train and test...
Encoding targets...
Done!


In [8]:
X_train_cp = X_train.copy()

In [9]:
X_test_cp = X_test.copy()

In [11]:
os_path = os.getcwd()
os_path_abs = '/'.join(os_path.split('/')[:-1])
model_path = os.path.join(os_path_abs,'WorkforceSentimentMonitoring/joblib_files')
model_path

'/Users/yunxizzy/code/theresa-sporn/WorkforceSentimentMonitoring/joblib_files'

In [65]:
model_nb_carrer = joblib.load(os.path.join(model_path, 'career-opportunities_nb.joblib'))
model_nb_carrer

Pipeline(steps=[('vectorizer',
                 ColumnTransformer(transformers=[('vectorizer',
                                                  TfidfVectorizer(analyzer='char',
                                                                  max_df=0.9794162818234204,
                                                                  min_df=0.017949846778863403,
                                                                  ngram_range=(1,
                                                                               5),
                                                                  stop_words='english'),
                                                  'review')])),
                ('classifier', MultinomialNB(alpha=0.0011498185515295503))])

In [30]:
fea_pipe = joblib.load(os.path.join(model_path, 'final_pipe.joblib'))
fea_pipe

Pipeline(steps=[('preprocessor', Preprocessor()),
                ('emo_extractor', EmotionScoresExtractor()),
                ('scaler', CustomMinMaxScaler()),
                ('engineer', FeatureEngineer())])

In [91]:
last_pipe = Pipeline([
    ('mnb', MultiNBFeaturesExtractor()),
    ('feature_pipe', fea_pipe)
])

In [92]:
last_pipe

Pipeline(steps=[('mnb', MultiNBFeaturesExtractor()),
                ('feature_pipe',
                 Pipeline(steps=[('preprocessor', Preprocessor()),
                                 ('emo_extractor', EmotionScoresExtractor()),
                                 ('scaler', CustomMinMaxScaler()),
                                 ('engineer', FeatureEngineer())]))])

In [96]:
# export the final feature pipeline
joblib.dump(last_pipe, 'joblib_files/last_pipe_test')

['joblib_files/last_pipe_test']

In [23]:
feature_pipe = joblib.load(os.path.join(model_path, 'feature_pipeline.joblib'))
feature_pipe

Pipeline(steps=[('first_pipe',
                 Pipeline(steps=[('preprocessor', Preprocessor()),
                                 ('emo_extractor', EmotionScoresExtractor()),
                                 ('scaler', CustomMinMaxScaler()),
                                 ('engineer', FeatureEngineer())])),
                ('mnb', MultiNBFeaturesExtractor())])

In [ ]:
X_train_fea = fea_pipe.fit_transform(X_train_cp)

In [73]:
X_train_cp

,review,work-balance_nb
0,A challenging place but also a good place to l...,0
1,Great Place to Start Your Career Microsoft's c...,0
2,"Over time offen. no coaching. Salary OK,work a...",1
3,"No one listens to their employees Decent pay, ...",0
4,Loved my time there! - standard silicon valley...,1
...,...,...
36658,Great experiece best life ever i mean 9 lakhs ...,0
36659,TRMS fun place to work and so much learning no...,1
36660,"Solid pay, people, and work hours Solid pay, p...",0
36661,Great Experience The people and the intelligen...,1


In [76]:
X_train_feature = get_mnb_features(X_train_cp)

In [85]:
X_train_feature_final = fea_pipe.fit_transform(X_train_feature)

100%|██████████| 36663/36663 [00:39<00:00, 939.30it/s]


In [83]:
X_train_fea_final = get_mnb_features(X_train_fea)

ValueError: Column ordering must be equal for fit and for transform when using the remainder keyword

In [86]:
X_train_feature_final

,review,work-balance_nb,culture-values_nb,career-opportunities_nb,comp-benefits_nb,senior-mgmt_nb,overall_nb,length,anger_score,anticipation_score,disgust_score,fear_score,joy_score,sadness_score,surprise_score,trust_score,subjectivity_review,polarity_review
0,a challenging place but also a good place to l...,0,1,1,1,1,1,0.138713,0.029633,0.386863,0.002665,0.047112,0.448213,0.036494,0.027958,0.371328,0.547872,0.199486
1,great place to start your career microsoft's c...,0,1,1,1,0,0,0.036392,0.039316,0.107264,0.029408,0.121476,0.020235,0.164043,0.033469,0.125077,0.569037,0.280303
2,over time offen no coaching salary ok work a a...,1,1,1,1,0,0,0.006593,0.000000,0.156931,0.000000,0.000000,0.104052,0.000000,0.000000,0.136223,0.295000,0.050000
3,no one listens to their employee decent pay gr...,0,1,1,1,0,1,0.007384,0.000000,0.051016,0.000000,0.000000,0.024837,0.000000,0.000000,0.116708,0.708333,0.483333
4,loved my time there standard silicon valley pe...,1,1,1,1,1,1,0.028481,0.085531,0.259345,0.086294,0.000000,0.403580,0.041743,0.029741,0.177813,0.559320,0.209728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36658,great experiece best life ever i mean lakh per...,0,1,1,1,0,0,0.006857,0.000000,0.000000,0.000000,0.000000,0.078623,0.000000,0.000000,0.000000,0.522917,0.547917
36659,trm fun place to work and so much learning not...,1,1,0,1,1,1,0.002637,0.000000,0.168088,0.000000,0.000000,0.218576,0.000000,0.000000,0.000000,0.200000,0.250000
36660,solid pay people and work hour solid pay peopl...,0,1,1,1,1,1,0.004747,0.000000,0.115616,0.000000,0.062652,0.056287,0.000000,0.000000,0.242719,0.400000,-0.166667
36661,great experience the people and the intelligen...,1,1,0,1,0,0,0.007120,0.010048,0.225995,0.000000,0.138804,0.194488,0.170489,0.000000,0.265587,0.568182,0.228788


# Rerun GridSearch of clf

In [33]:
from WorkforceSentimentMonitoring.trainer import MultiNBFeaturesExtractor
from WorkforceSentimentMonitoring.feature_engineering import get_mnb_features

In [34]:
mnb = MultiNBFeaturesExtractor()

In [14]:
X_train_cp = X_train.copy()
X_test_cp = X_test.copy()

In [15]:
X_test_cp

,review
0,A good place to work in retail. Transitioning ...
1,process assistant good team members and good f...
2,Great opportunities and competitive compensati...
3,Largest Operations Company Great if you have n...
4,Good for balance work and life 1. You will suf...
...,...
15709,"Google Colleagues, challenges, business prospe..."
15710,"Software Engineer, Tools & Infrastructure Good..."
15711,Great culture. The best work culture and a gre...
15712,Amazon Amazon has amazing principles to help t...


In [29]:
X_train_cp

,review
0,It all depends on who you work under Really gr...
1,One of the Best Apple is a tough place to work...
2,Great place to work with The speed at which wo...
3,"It’s Ok Benefits, Discounts, Schedule Preferen..."
4,"Work hard, have no fun Smart people, company i..."
...,...
36659,"Unprofessional, unrespectful environment - Goo..."
36660,Machine Learning Researcher Most innovating co...
36661,"Good people, Good Products, Overall negative e..."
36662,"Amazing Amazon Good culture, good salary, grea..."


In [20]:
test_pipe = joblib.load(os.path.join(model_path, 'last_pipe_test'))
test_pipe

Pipeline(steps=[('mnb', MultiNBFeaturesExtractor()),
                ('feature_pipe',
                 Pipeline(steps=[('preprocessor', Preprocessor()),
                                 ('emo_extractor', EmotionScoresExtractor()),
                                 ('scaler', CustomMinMaxScaler()),
                                 ('engineer', FeatureEngineer())]))])

In [22]:
X_test_cp_final = test_pipe.fit_transform(X_test_cp.iloc[:,:1])

ValueError: Number of features of the input must be equal to or greater than that of the fitted transformer. Transformer n_features is 18 and input n_features is 1.

In [24]:
X_test_cp_final = feature_pipe.fit_transform(X_test_cp.iloc[:,:1])

100%|██████████| 15714/15714 [00:16<00:00, 932.31it/s]


ValueError: Number of features of the input must be equal to or greater than that of the fitted transformer. Transformer n_features is 18 and input n_features is 12.

In [31]:
X_train_final = fea_pipe.fit_transform(X_train_cp)

100%|██████████| 36664/36664 [00:37<00:00, 974.97it/s] 


In [ ]:
X_test_fea = fea_pipe.fit_transform(X_train_cp)

In [35]:
X_train_final

,review,length,anger_score,anticipation_score,disgust_score,fear_score,joy_score,sadness_score,surprise_score,trust_score,subjectivity_review,polarity_review
0,it all depends on who you work under really gr...,0.030854,0.029244,0.222245,0.000000,0.016229,0.057320,0.000000,0.000000,0.189453,0.375641,0.124359
1,one of the best apple is a tough place to work...,0.023734,0.023644,0.066769,0.025039,0.000000,0.443397,0.057742,0.095773,0.133583,0.580159,0.263492
2,great place to work with the speed at which wo...,0.018460,0.000000,0.081649,0.000000,0.043482,0.052051,0.000000,0.011877,0.333112,0.510240,0.280819
3,it is ok benefit discount schedule preference ...,0.010285,0.000000,0.207595,0.000000,0.000000,0.232136,0.086526,0.108984,0.110595,0.542857,0.228571
4,work hard have no fun smart people company is ...,0.068829,0.117591,0.313718,0.118540,0.161596,0.476503,0.286315,0.108298,0.230866,0.550040,0.103225
...,...,...,...,...,...,...,...,...,...,...,...,...
36659,unprofessional unrespectful environment good t...,0.020306,0.186666,0.259883,0.051316,0.096641,0.113539,0.140292,0.053903,0.311046,0.563333,0.101667
36660,machine learning researcher most innovating co...,0.006065,0.000000,0.029445,0.000000,0.000000,0.038942,0.000000,0.016837,0.085060,0.400000,0.325000
36661,good people good product overall negative envi...,0.020306,0.101490,0.237630,0.153940,0.365110,0.184206,0.148061,0.021485,0.150833,0.429464,0.179464
36662,amazing amazon good culture good salary great ...,0.002110,0.160632,0.325239,0.000000,0.124005,0.446149,0.000000,0.084355,0.289398,0.712500,0.700000


In [39]:
X_train_features = get_mnb_features(X_train_final)

ValueError: Number of features of the input must be equal to or greater than that of the fitted transformer. Transformer n_features is 18 and input n_features is 12.

In [104]:
# search with SGD
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd = SGDClassifier()
param = {
    'alpha': stats.loguniform(0.00001, 1),
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'penalty': ['l1', 'l2']#,
    #'class_weight': ['balanced', 'None']
         }
sgd_search = RandomizedSearchCV(sgd, param, scoring='accuracy', n_iter=100, cv=10, n_jobs=-1)

In [ ]:
# final test
sgd_final_balance = sgd_search.fit(X_train_final.drop(columns='review'), y_train['work-balance'])
sgd_final_balance.best_score_, \
sgd_final_balance.best_estimator_

In [ ]:
# final test
sgd_final_culture = sgd_search.fit(X_train_final.drop(columns='review'), y_train['culture-values'])
sgd_final_culture.best_score_, \
sgd_final_culture.best_estimator_

In [ ]:
# final test
sgd_final_benefits = sgd_search.fit(X_train_final.drop(columns='review'), y_train['comp-benefits'])
sgd_final_benefits.best_score_, \
sgd_final_benefits.best_estimator_

In [ ]:
# final test
sgd_final_mgmt = sgd_search.fit(X_train_final.drop(columns='review'), y_train['senior-mgmt'])
sgd_final_mgmt.best_score_, \
sgd_final_mgmt.best_estimator_

In [ ]:
# final test
sgd_final_career = sgd_search.fit(X_train_final.drop(columns='review'), y_train['career-opportunities'])
sgd_final_career.best_score_, \
sgd_final_career.best_estimator_

In [ ]:
# final test
sgd_final_overall = sgd_search.fit(X_train_final.drop(columns='review'), y_train['overall'])
sgd_final_overall.best_score_, \
sgd_final_overall.best_estimator_